# IMPORTING LIB

In [9]:
!pip install pyroomacoustics
!pip install samplerate

  Using cached samplerate-0.2.1.tar.gz (21 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for samplerate (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for samplerate
Failed to build samplerate
ERROR: Could not build wheels for samplerate, which is required to install pyproject.toml-based projects


In [10]:
import os
import numpy as np
import pyroomacoustics as pra
from scipy.io import wavfile
from scipy.signal import fftconvolve
import matplotlib.pyplot as plt
import librosa
import IPython
from IPython.display import Audio

ModuleNotFoundError: No module named 'samplerate'

# SOURCE GENERATION

In [ ]:
# Load the audio file
fs, voice1 = wavfile.read("arctic_a0010.wav")

# Normalize the audio signal
voice1 = voice1 / abs(voice1).max()

# Listen to the normalized audio
Audio(voice1, rate=fs)

In [ ]:
# Plot the waveform
plt.figure(figsize=(12, 6))
plt.subplot(3, 1, 1)
plt.plot(np.arange(len(voice1)) / fs, voice1)
plt.title('Waveform')
plt.xlabel('Time (s)')
plt.ylabel('Amplitude')

# Plot the spectrogram
plt.subplot(3, 1, 2)
plt.specgram(voice1, Fs=fs, cmap='viridis')
plt.title('Spectrogram')
plt.xlabel('Time (s)')
plt.ylabel('Frequency (Hz)')

# Plot the histogram
plt.subplot(3, 1, 3)
plt.hist(voice1, bins=100)
plt.title('Histogram')
plt.xlabel('Amplitude')
plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

In [ ]:
# Generate colored noise with the same length as the audio file
noise = np.random.randn(len(voice1))

# Normalize the audio signal
noise = noise / abs(voice1).max()

# Listen to the colored noise
Audio(noise, rate=fs)

In [ ]:
# Plot the waveform
plt.figure(figsize=(12, 6))
plt.subplot(3, 1, 1)
plt.plot(np.arange(len(noise)) / fs, noise)
plt.title('Waveform')
plt.xlabel('Time (s)')
plt.ylabel('Amplitude')

# Plot the spectrogram
plt.subplot(3, 1, 2)
plt.specgram(noise, Fs=fs, cmap='viridis')
plt.title('Spectrogram')
plt.xlabel('Time (s)')
plt.ylabel('Frequency (Hz)')

# Plot the histogram
plt.subplot(3, 1, 3)
plt.hist(noise, bins=100)
plt.title('Histogram')
plt.xlabel('Amplitude')
plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

# ROOM CREATION

In [ ]:
# Define the corners of a 2D room as an array of (x, y) coordinates
corners = np.array([[0,0], [0,3], [5,3], [5,1], [3,1], [3,0]]).T  # [x,y]

# Create a 2D room object from the given corners
# max_order: Maximum reflection order for ray tracing
# materials: Defines the acoustic properties of the walls (absorption coefficient, reflection coefficient)
room = pra.Room.from_corners(corners, max_order=3, materials=pra.Material(0.2, 0.15), ray_tracing=True, air_absorption=True)

# Set the parameters for ray tracing
# receiver_radius: Radius of the receiver for computing the visibility polygon
# n_rays: Number of rays to use for ray tracing
# energy_thres: Threshold energy for ray termination
room.set_ray_tracing(receiver_radius=0.5, n_rays=10000, energy_thres=1e-5)

# Extrude the 2D room to create a 3D room with a specified height
# materials: Defines the acoustic properties of the walls in 3D (absorption coefficient, reflection coefficient)
room.extrude(3., materials=pra.Material(0.2, 0.15))

# Plot the room
fig, ax = room.plot()
ax.set_xlim([0, 5])  # Set x-axis limits
ax.set_ylim([0, 5])  # Set y-axis limits
ax.set_zlim([0, 4])  # Set z-axis limits
ax.set_xlabel('X')  # Label for x-axis
ax.set_ylabel('Y')  # Label for y-axis
ax.set_zlabel('Z')  # Label for z-axis

In [ ]:
# Define the location of sources within the room
source1_loc = [1., 1., 1.]
source2_loc = [1., 2., 1.]

# Add source 1 to the room with a specified signal (voice1)
room.add_source(source1_loc, signal=voice1)

# Add source 2 to the room with a specified signal (noise)
room.add_source(source2_loc, signal=noise)

# Define the positions of two microphones in 3D space
# Ensure each microphone position is specified in 3D (x, y, z) coordinates
mic_positions = np.array([[3.5, 3.6], [2., 2.], [0.5,  0.5]])

# Add the microphone array to the room
room.add_microphone_array(pra.MicrophoneArray(mic_positions, fs))

# Plot the room with sources and microphone array
fig, ax = room.plot()

# Set limits for the plot's x, y, and z axes
ax.set_xlim([0, 5])  # Set x-axis limits
ax.set_ylim([0, 5])  # Set y-axis limits
ax.set_zlim([0, 4])  # Set z-axis limits

# Label the axes for clarity
ax.set_xlabel('X')  # Label for x-axis
ax.set_ylabel('Y')  # Label for y-axis
ax.set_zlabel('Z')  # Label for z-axis


In [ ]:
# Plot the room impulse response (RIR)
room.plot_rir()

# Get the current figure
fig = plt.gcf()

# Set the size of the figure for better visualization
fig.set_size_inches(20, 10)

In [ ]:
room.simulate()
print(room.mic_array.signals.shape)

# Nuova sezione